In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
from langchain_groq import ChatGroq

In [22]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key = groq_api_key , model_name = "Llama3-8b-8192",temperature=0.5)


In [18]:
os.environ["TAVILY_API_KEY"] = ""

tools = [TavilySearchResults(max_results=1)]


In [19]:
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [20]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:

agent = create_openai_functions_agent(llm, tools, prompt)

In [24]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://python.langchain.com/v0.2/docs/introduction/', 'content': "LangChain is a framework for developing applications powered by large language models (LLMs). Learn how to use LangChain's open-source libraries, components, and integrations to build, deploy, and evaluate LLM applications and agents."}]LangChain is a framework for developing applications powered by large language models (LLMs).

> Finished chain.


{'input': 'what is LangChain?',
 'output': 'LangChain is a framework for developing applications powered by large language models (LLMs).'}

In [26]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

In [59]:
# main.py

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_groq import ChatGroq
import sqlite3
from langchain.tools import Tool
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq language model
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="gemma2-9b-It",
    temperature=0.5
)

# Connect to the SQLite database
conn = sqlite3.connect("db.sqlite")

def run_sqlite_query(query):
    """Execute a SQLite query and return the results."""
    c = conn.cursor()
    c.execute(query)
    return c.fetchall()

# Define the tool for running SQLite queries
run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a SQLite query.",
    func=run_sqlite_query
)

# Define the chat prompt template
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

# Initialize the agent with the LLM, prompt, and tools
agent = create_openai_functions_agent(
    llm=llm,
    prompt=prompt,
    tools=[run_query_tool]
)

# Initialize the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=[run_query_tool]
)

# Execute the agent with the input query
response = agent_executor.invoke({"input": "How many users are in the database ?"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT COUNT(*) FROM users`


[(2000,)]2000

> Finished chain.
{'input': 'How many users are in the database ?', 'output': '2000'}


In [74]:
# main.py

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_groq import ChatGroq
import sqlite3
from langchain.tools import Tool
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import initialize_agent, AgentType
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq language model
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Llama3-8b-8192",
    temperature=0.5
)

# Connect to the SQLite database
conn = sqlite3.connect("db.sqlite")

def run_sqlite_query(query):
    """Execute a SQLite query and return the results."""
    c = conn.cursor()
    c.execute(query)
    return c.fetchall()

# Define the tool for running SQLite queries
run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a SQLite query.",
    func=run_sqlite_query
)

# Define the chat prompt template
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

# Initialize the agent with the LLM, prompt, and tools
agent = create_openai_functions_agent(
    llm=llm,
    prompt=prompt,
    tools=[run_query_tool]
)

# Initialize the agent executor
agent_executor = initialize_agent(
    llm = llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    tools=[run_query_tool]
)

# Explicit input for the agent
input_data = {"input": "How many users are in the database?"}

# Execute the agent with the input query
response = agent_executor.invoke({"input": "How many users are in the database?"})
print(response)




> Entering new AgentExecutor chain...
Thought: I need to query the database to get the count of users.

Action: run_sqlite_query
Action Input: SELECT COUNT(*) FROM users;
Observation: [(2000,)]
Thought:Here's the response:

Question: How many users are in the database?
Thought: I need to query the database to get the count of users.

Action: run_sqlite_query
Action Input: SELECT COUNT(*) FROM users;
Observation: [(2000,)]
Thought:Thought: I already know the answer from the previous query, so I can just report it.

Final Answer: 2000

> Finished chain.
{'input': 'How many users are in the database?', 'output': '2000'}


In [76]:
response = agent_executor.invoke({"input": "please Show the table names in the databse"})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to retrieve the table names from the database. I can do this by running a query that retrieves the names of all tables in the database.

Action: run_sqlite_query
Action Input: "SELECT name FROM sqlite_master WHERE type='table';"
Observation: [('users',), ('addresses',), ('products',), ('carts',), ('orders',), ('order_products',)]
Thought:Thought: I've retrieved the table names from the database. Now I need to display them in a readable format.

Action: None

Action Input: None

Observation: None is not a valid tool, try one of [run_sqlite_query].
Thought:Let's continue.

Thought: I need to display the table names in a readable format. I can use the observation from the previous action and print it out.

Action: None

Action Input: None

Observation: None is not a valid tool, try one of [run_sqlite_query].
Thought:Here's the continuation:

Action: print
Action Input: Observation
Observation: print is not a valid tool, try one of [r

In [77]:
response['output']

'The table names in the database are: users, addresses, products, carts, orders, order_products.'

In [4]:
from langchain.pydantic_v1 import BaseModel
from langchain.tools import StructuredTool

ImportError: cannot import name 'StracturedTool' from 'langchain.tools' (c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\langchain\tools\__init__.py)

In [1]:
from pydantic import BaseSchema 

ImportError: cannot import name 'BaseSchema' from 'pydantic' (c:\Users\s\miniconda3\envs\langchainenv\lib\site-packages\pydantic\__init__.cp310-win_amd64.pyd)

In [69]:
# main.py

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_groq import ChatGroq
import sqlite3
from langchain.tools import Tool
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
# from langchain_core.tools import InjectedToolArg
# from langgraph.prebuilt import create_react_agent

import os
#from langchain_core.tools import InjectedToolArg
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq language model
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Llama3-8b-8192",
    temperature=0.5
)

# Connect to the SQLite database
conn = sqlite3.connect("db.sqlite3")

def run_sqlite_query(query):
    """Execute a SQLite query and return the results."""
    c = conn.cursor()
    c.execute(query)
    return c.fetchall()

# Define the tool for running SQLite queries
run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a SQLite query.",
    func=run_sqlite_query
)

# Define the chat prompt template
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

# Initialize the agent with the LLM, prompt, and tools
agent = create_openai_functions_agent(
    llm=llm,
    prompt=prompt,
    tools=[run_query_tool]
)

# Initialize the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=[run_query_tool]
)

# Explicit input for the agent
input_data = {"input": "How many users are in the database?"}

# Execute the agent with the input query
response = agent_executor.invoke(input_data)
print(response)




> Entering new AgentExecutor chain...
Tool use failed: no tool can be called with name get_user_count

> Finished chain.
{'input': 'How many users are in the database?', 'output': 'Tool use failed: no tool can be called with name get_user_count'}


In [36]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

from langchain.agents import AgentExecutor ,OpenAIFunctionsAgent

prompt = ChatPromptTemplate(
    messages = [
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')

    ]
)

In [40]:
tools = [run_query_tool]

In [41]:
agent = create_openai_functions_agent(llm = llm,
                             prompt = prompt,
                             tools = [run_query_tool]
                             )

In [44]:
agent_executor = AgentExecutor(
    agent = agent ,
    Verbose = True ,
    tools = [run_query_tool]
)

In [46]:
agent_executor.invoke({"input": "what is LangChain?"})

IndexError: list index out of range

In [ ]:
""" 
In LangChain, callbacks and callback handlers are mechanisms that allow you to track, log, and manipulate the execution flow of chains, agents, and models. These tools are particularly useful for debugging, logging, monitoring, and customizing the behavior of your language models or other components in the LangChain framework.

1. What is a Callback?
A callback is essentially a function or a set of functions that gets called at specific points during the execution of a chain, agent, or model. These points can include:

Before an action is taken (e.g., before a tool is invoked).
After an action is taken (e.g., after a tool returns a result).
On error (e.g., when an error occurs during execution).
On chain start or end (e.g., when a chain begins or finishes processing).
2. What is a Callback Handler?
A callback handler is an object that contains one or more callback functions. LangChain uses these handlers to manage and trigger callbacks during execution. You can attach multiple callback handlers to a chain or agent, and each handler can define custom logic for each of the possible callback events.

3. Why Use Callbacks and Callback Handlers?
Callbacks and callback handlers allow you to:

Monitor Execution: Log inputs, outputs, and intermediate steps.
Debug: Track where things go wrong during execution.
Modify Behavior: Adjust the execution flow based on specific conditions.
Track Performance: Measure the time taken by different parts of the chain or agent.
4. Examples
Basic Example: Logging with a Callback Handler
Suppose you want to log the start and end of a chain's execution:

python
Copy code
from langchain.callbacks import BaseCallbackHandler

class MyLoggingCallbackHandler(BaseCallbackHandler):
    def on_chain_start(self, serialized, inputs, **kwargs):
        print(f"Chain started with inputs: {inputs}")

    def on_chain_end(self, outputs, **kwargs):
        print(f"Chain ended with outputs: {outputs}")

# Initialize your handler
my_logging_handler = MyLoggingCallbackHandler()

# Attach the handler to a chain or agent
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=[run_query_tool],
    callbacks=[my_logging_handler]  # Attach the callback handler here
)

# Execute the agent with the input query
response = agent_executor.invoke({"input": "How many users are in the database?"})
This will print logs when the chain starts and ends, making it easier to follow the flow of execution.

Advanced Example: Timing Execution Steps
If you want to measure how long each part of the chain or agent takes to execute:

python
Copy code
import time
from langchain.callbacks import BaseCallbackHandler

class TimingCallbackHandler(BaseCallbackHandler):
    def on_chain_start(self, serialized, inputs, **kwargs):
        self.start_time = time.time()
        print("Chain started.")

    def on_chain_end(self, outputs, **kwargs):
        end_time = time.time()
        elapsed_time = end_time - self.start_time
        print(f"Chain ended. Time taken: {elapsed_time:.2f} seconds")

# Initialize your handler
timing_handler = TimingCallbackHandler()

# Attach the handler to a chain or agent
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=[run_query_tool],
    callbacks=[timing_handler]  # Attach the callback handler here
)

# Execute the agent with the input query
response = agent_executor.invoke({"input": "How many users are in the database?"})
This callback handler will measure and print how long the entire chain or agent took to execute.

5. Built-in Callback Handlers
LangChain provides some built-in callback handlers, such as:

StreamingStdOutCallbackHandler: Streams output to standard output.
AsyncCallbackHandler: Handles callbacks asynchronously.
TracingCallbackHandler: Integrates with tracing systems to monitor execution.
You can also create your custom callback handlers by subclassing BaseCallbackHandler.

6. Using Multiple Callback Handlers
You can attach multiple callback handlers to a single chain or agent. Each handler will be invoked according to the events it subscribes to. This is useful for combining different types of monitoring and logging behaviors.

Conclusion
Callbacks and callback handlers in LangChain are powerful tools for managing and customizing the execution of chains and agents. They offer a way to insert custom logic at different stages of the execution process, making it easier to debug, log, and control the behavior of your applications.
"""